## Operator overloading: doing it right



### When x and +x are not equal


In [7]:
# +X 生成了一个新的对象
import decimal

ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')

one_third

Decimal('0.3333333333333333333333333333333333333333')

In [3]:
one_third == +one_third

True

In [4]:
ctx.prec = 28
one_third == +one_third

False

In [5]:
+one_third

Decimal('0.3333333333333333333333333333')

In [6]:
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [8]:
# example 2

import collections

ct = collections.Counter('asdfghgfdsdfgf')
ct

Counter({'a': 1, 'd': 3, 'f': 4, 'g': 3, 'h': 1, 's': 2})

In [9]:
ct['a'] = 0
ct['d'] = -2

In [10]:
ct

Counter({'a': 0, 'd': -2, 'f': 4, 'g': 3, 'h': 1, 's': 2})

In [12]:
# 返回了一个新的对象
+ct

Counter({'f': 4, 'g': 3, 'h': 1, 's': 2})

### Overloading + for vector addition


In [13]:
# chapter 10 中的 Vecter 例子

from array import array
import math
import reprlib

class Vecter:
    
    typecode = 'd'
    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vecter({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

In [14]:
# 继承构建本章所需

class Vecter13(Vecter):
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))
    
    def __neg__(self):
        return Vecter13(-x for x in self)
    
    def __pos__(self):
        return Vecter13(self)
    
    

In [16]:
v1 = Vecter13([1,2,3])
abs(v1)

3.7416573867739413

In [17]:
-v1

Vecter([-1.0, -2.0, -3.0])

In [18]:
+v1

Vecter([1.0, 2.0, 3.0])

In [19]:
# 构建加法运算
import itertools

class VecterAdd(Vecter13):
    
    def __add__(self, other):
        
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return VecterAdd(a+b for a, b in pairs)
    
    

In [20]:
v2 = VecterAdd([1, 2, 3])
v3 = VecterAdd([4, 5, 6])

v2 + v3

Vecter([5.0, 7.0, 9.0])

In [21]:
v2 + [7,8,9]

Vecter([8.0, 10.0, 12.0])

In [22]:
v2 + VecterAdd([3,4,5,6,7])

Vecter([4.0, 6.0, 8.0, 6.0, 7.0])

In [23]:
v2 + Vecter([5,6,7])

Vecter([6.0, 8.0, 10.0])

In [25]:
[3,4,5] + v2

TypeError: can only concatenate list (not "VecterAdd") to list

In [26]:
# 针对以上错误，添加 __radd__ 方法即可

class VecterAdd２(Vecter13):
    
    def __add__(self, other):
        
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return VecterAdd(a+b for a, b in pairs)
    
    def __radd__(self, other):
        return self + other
    

In [27]:
[1,2,3] + VecterAdd2([4,5,6])

Vecter([5.0, 7.0, 9.0])

\_\_add\_\_　的flowchart:
 
![](Selection_003.jpg)

### Overloading * for scalar multiplication


In [31]:
# 乘法的实现

class VecterMul(Vecter):
    
    def __mul__(self, scalar):
        return VecterAdd2([x*scalar for x in self])
    
    def __rmul__(self, scalar):
        return self * scalar

In [45]:
v5 = VecterMul([1,3,5])

In [36]:
v5 * 2

Vecter([2.0, 6.0, 10.0])

In [37]:
2 * v5

Vecter([2.0, 6.0, 10.0])

In [38]:
v5 * True

Vecter([1.0, 3.0, 5.0])

In [39]:
isinstance(v5, Vecter)

True

In [40]:
isinstance(v5, VecterMul)

True

In [46]:
v5 *= 2
v5

Vecter([2.0, 6.0, 10.0])